# Install Required Libraries
Install necessary libraries such as OpenCV, TensorFlow/PyTorch, Pandas, and Matplotlib/Plotly.

In [5]:
!pip install opencv-python-headless torch pandas matplotlib plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 6.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 9.2 MB/s eta 0:00:00a 0:00:01


# Import Required Libraries
Import necessary libraries such as OpenCV, TensorFlow/PyTorch, Pandas, and Matplotlib/Plotly.

In [2]:
import cv2  # OpenCV for frame extraction and image processing
import torch  # PyTorch for bird classification
import pandas as pd  # Pandas for data storage and processing
import matplotlib.pyplot as plt  # Matplotlib for visualization
import plotly.express as px  # Plotly for interactive visualization

ModuleNotFoundError: No module named 'cv2'

# Frame Extraction
Load a time-lapse video, extract frames at a configurable interval, and save the extracted frames for processing.

In [ ]:
import cv2  # OpenCV for frame extraction and image processing

# Function to extract frames from video
def extract_frames(video_path, output_folder, interval=1):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        return
    
    # Get the total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Frame counter
    frame_count = 0
    
    # Loop through the video frames
    while cap.isOpened():
        ret, frame = cap.read()
        
        # If frame is read correctly
        if ret:
            # Save frame at the specified interval
            if frame_count % interval == 0:
                frame_filename = f"{output_folder}/frame_{frame_count}.jpg"
                cv2.imwrite(frame_filename, frame)
            
            frame_count += 1
        else:
            break
    
    # Release the video capture object
    cap.release()
    print(f"Extracted {frame_count // interval} frames from the video.")

# Example usage
video_path = 'path_to_your_video.mp4'
output_folder = 'extracted_frames'
extract_frames(video_path, output_folder, interval=30)  # Extract 1 frame per second (assuming 30 fps video)

# Bird Detection
Use a pre-trained object detection model (e.g., YOLOv8 or Faster R-CNN) to locate birds in each frame and draw bounding boxes around detected birds.

In [ ]:
import cv2  # OpenCV for frame extraction and image processing
import torch  # PyTorch for bird classification

# Load pre-trained YOLOv8 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # Replace 'yolov5s' with 'yolov8' if available

# Function to detect birds in a frame
def detect_birds(frame):
    # Perform inference
    results = model(frame)
    
    # Extract bounding boxes and labels
    boxes = results.xyxy[0].numpy()  # Bounding boxes
    labels = results.names  # Class labels
    
    return boxes, labels

# Function to draw bounding boxes on the frame
def draw_bounding_boxes(frame, boxes, labels):
    for box in boxes:
        x1, y1, x2, y2, conf, cls = box
        label = labels[int(cls)]
        
        # Draw rectangle
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        
        # Put label
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame

# Example usage
frame_path = 'extracted_frames/frame_0.jpg'
frame = cv2.imread(frame_path)

# Detect birds in the frame
boxes, labels = detect_birds(frame)

# Draw bounding boxes on the frame
frame_with_boxes = draw_bounding_boxes(frame, boxes, labels)

# Save the frame with bounding boxes
cv2.imwrite('detected_birds_frame_0.jpg', frame_with_boxes)

# Species Classification
Use a deep learning model trained on bird species to classify detected birds and implement color-based filtering for species differentiation if necessary.

In [ ]:
import torch
import torchvision.transforms as transforms
from PIL import Image

# Load pre-trained bird species classification model
# Assuming a model trained on bird species is available
classification_model = torch.load('path_to_your_bird_classification_model.pth')
classification_model.eval()

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to classify bird species in a detected bounding box
def classify_bird_species(frame, boxes):
    species_list = []
    for box in boxes:
        x1, y1, x2, y2, conf, cls = box
        bird_crop = frame[int(y1):int(y2), int(x1):int(x2)]
        bird_image = Image.fromarray(cv2.cvtColor(bird_crop, cv2.COLOR_BGR2RGB))
        bird_tensor = transform(bird_image).unsqueeze(0)
        
        with torch.no_grad():
            outputs = classification_model(bird_tensor)
            _, predicted = torch.max(outputs, 1)
            species = predicted.item()
            species_list.append(species)
    
    return species_list

# Example usage
frame_path = 'extracted_frames/frame_0.jpg'
frame = cv2.imread(frame_path)

# Detect birds in the frame
boxes, labels = detect_birds(frame)

# Classify bird species in the detected bounding boxes
species_list = classify_bird_species(frame, boxes)

# Print the detected species
print("Detected species:", species_list)

# Counting & Data Storage
Track unique birds to avoid duplicate counting across frames and store species counts in a CSV or JSON file for analysis.

In [ ]:
import json

# Function to count unique birds and store species counts in a CSV or JSON file
def count_and_store_bird_data(frames_folder, output_file, output_format='csv'):
    bird_data = []
    
    # Loop through the extracted frames
    for frame_filename in os.listdir(frames_folder):
        frame_path = os.path.join(frames_folder, frame_filename)
        frame = cv2.imread(frame_path)
        
        # Detect birds in the frame
        boxes, labels = detect_birds(frame)
        
        # Classify bird species in the detected bounding boxes
        species_list = classify_bird_species(frame, boxes)
        
        # Track unique birds and count occurrences
        for species in species_list:
            bird_data.append({
                'frame': frame_filename,
                'species': species,
                'timestamp': frame_filename.split('_')[1].split('.')[0]  # Assuming frame filenames are in the format 'frame_<timestamp>.jpg'
            })
    
    # Convert bird data to a DataFrame
    bird_df = pd.DataFrame(bird_data)
    
    # Store data in the specified format
    if output_format == 'csv':
        bird_df.to_csv(output_file, index=False)
    elif output_format == 'json':
        bird_df.to_json(output_file, orient='records', lines=True)
    else:
        print("Unsupported output format. Please choose 'csv' or 'json'.")

# Example usage
frames_folder = 'extracted_frames'
output_file = 'bird_data.csv'
count_and_store_bird_data(frames_folder, output_file, output_format='csv')

# Visualization
Generate a time-series plot showing species counts over time and optionally create a dashboard for real-time visualization.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px

# Function to generate a time-series plot showing species counts over time
def plot_species_counts(csv_file):
    # Load the bird data from CSV
    bird_df = pd.read_csv(csv_file)
    
    # Convert timestamp to datetime
    bird_df['timestamp'] = pd.to_datetime(bird_df['timestamp'], unit='s')
    
    # Group by timestamp and species to count occurrences
    species_counts = bird_df.groupby(['timestamp', 'species']).size().reset_index(name='count')
    
    # Pivot the data for plotting
    species_pivot = species_counts.pivot(index='timestamp', columns='species', values='count').fillna(0)
    
    # Plot the data using Matplotlib
    species_pivot.plot(kind='line', figsize=(12, 6))
    plt.title('Bird Species Counts Over Time')
    plt.xlabel('Time')
    plt.ylabel('Count')
    plt.legend(title='Species')
    plt.grid(True)
    plt.show()

# Function to create an interactive dashboard for real-time visualization
def create_dashboard(csv_file):
    # Load the bird data from CSV
    bird_df = pd.read_csv(csv_file)
    
    # Convert timestamp to datetime
    bird_df['timestamp'] = pd.to_datetime(bird_df['timestamp'], unit='s')
    
    # Group by timestamp and species to count occurrences
    species_counts = bird_df.groupby(['timestamp', 'species']).size().reset_index(name='count')
    
    # Create an interactive line plot using Plotly
    fig = px.line(species_counts, x='timestamp', y='count', color='species', title='Bird Species Counts Over Time')
    fig.show()

# Example usage
csv_file = 'bird_data.csv'
plot_species_counts(csv_file)
create_dashboard(csv_file)